In [1]:
from __future__ import print_function
from __future__ import division

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import statsmodels.api as sm

from statsmodels.tools import eval_measures
import statsmodels.formula.api as smf

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

%matplotlib inline
plt.rcParams['figure.figsize']= (8,8)

from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
train_features = pd.read_csv('./data/dengue_features_train.csv')

train_labels = pd.read_csv('./data/dengue_labels_train.csv')
train_labels = train_labels.total_cases

test_features = pd.read_csv('./data/dengue_features_test.csv')

submission = pd.read_csv('./data/submission_format.csv')

In [3]:
train_set = pd.concat([train_labels, train_features], axis=1)
train_sj = train_set[train_set.city == 'sj'].copy()
train_iq = train_set[train_set.city == 'iq'].copy()

ytrain_sj = train_sj['total_cases'].copy()
ytrain_iq = train_iq['total_cases'].copy()

test_sj = test_features[train_set.city == 'sj'].copy()
test_iq = test_features[train_set.city == 'iq'].copy()


In [4]:
keys = ['weekofyear','city', 'year']

sj_features = ['reanalysis_dew_point_temp_k', 'reanalysis_precip_amt_kg_per_m2', 'reanalysis_specific_humidity_g_per_kg', 
            'reanalysis_avg_temp_k',  'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k']

iq_features = ['reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k', 'reanalysis_avg_temp_k', 'precipitation_amt_mm', 
                 'reanalysis_dew_point_temp_k', 'reanalysis_air_temp_k', 'reanalysis_relative_humidity_percent',
                 'reanalysis_precip_amt_kg_per_m2', 'ndvi_se', 'ndvi_ne', 'station_precip_mm']

sj_all_features = list( set(sj_features).union(set(keys)) )
iq_all_features = list( set(iq_features).union(set(keys)) )


In [5]:
train_sj = train_sj[sj_all_features]
train_iq = train_iq[iq_all_features]
test_sj = test_sj[sj_all_features]
test_iq = test_iq[iq_all_features]


In [6]:
train_sj.fillna(method='ffill', inplace=True)
train_iq.fillna(method='ffill', inplace=True)
test_sj.fillna(method='ffill', inplace=True)
test_iq.fillna(method='ffill', inplace=True)

In [7]:
def normalize(feature):
    return (feature - feature.mean()) / feature.std()


In [8]:
train_sj[sj_features] = train_sj[sj_features].apply(normalize, axis=0)
train_iq[iq_features] = train_iq[iq_features].apply(normalize, axis=0)
test_sj[sj_features] = test_sj[sj_features].apply(normalize, axis=0)
test_iq[iq_features] = test_iq[iq_features].apply(normalize, axis=0)


In [9]:
Xtrn_sj, Xtst_sj, Ytrn_sj, Ytst_sj = train_test_split(train_sj, ytrain_sj, test_size=0.2, stratify=train_sj.weekofyear)
Xtrn_iq, Xtst_iq, Ytrn_iq, Ytst_iq = train_test_split(train_iq, ytrain_iq, test_size=0.2, stratify=train_iq.weekofyear)


In [10]:
sj_sub = Xtst_sj[['city','year']]
iq_sub = Xtst_iq[['city','year']]

In [11]:
Xtrn_sj = Xtrn_sj.drop(['city','year'],axis=1)
Xtst_sj = Xtst_sj.drop(['city','year'],axis=1)
Xtrn_iq = Xtrn_iq.drop(['city','year'],axis=1)
Xtst_iq = Xtst_iq.drop(['city','year'],axis=1)

In [12]:
def grid_search_cross_val(reg, X, Y, param_grid, scoring='neg_mean_absolute_error'):
    grid = GridSearchCV(reg, param_grid=param_grid, scoring=scoring)
    grid.fit(X, Y)
    print( "Best score: {}".format(np.abs(grid.best_score_)))
    print( "Best params: {}".format(grid.best_params_))

In [13]:
%%time
reg = RandomForestRegressor(random_state=67)

param_grid = [
    {
      'n_estimators': [10, 30, 100, 300, 500, 1000], 
      'max_depth': [3, 5, 7, None]
    } 
]

print( "San Juan")
grid_search_cross_val(reg, Xtrn_sj, Ytrn_sj, param_grid)

San Juan
Best score: 25.376366785062128
Best params: {'max_depth': 3, 'n_estimators': 10}
CPU times: user 31.2 s, sys: 330 ms, total: 31.5 s
Wall time: 31.7 s


In [14]:
def get_best_iq_model(train, test):
    model_formula = "total_cases ~ 1 + " \
                    "reanalysis_min_air_temp_k + reanalysis_avg_temp_k + reanalysis_max_air_temp_k + " \
                    "precipitation_amt_mm + reanalysis_dew_point_temp_k + reanalysis_air_temp_k + " \
                    "reanalysis_relative_humidity_percent + reanalysis_precip_amt_kg_per_m2 + " \
                    "ndvi_se + ndvi_ne + station_precip_mm"
    
    grid = 10 ** np.arange(-8, -3, dtype=np.float64)             
    best_alpha = []
    best_score = 1000
    
    for alpha in grid:
        model = smf.glm(formula=model_formula, data=train, family=sm.families.NegativeBinomial(alpha=alpha))

        results = model.fit()
        predictions = results.predict(test).astype(int)
        score = eval_measures.meanabs(predictions, test.total_cases)

        if score < best_score:
            best_alpha = alpha
            best_score = score

    print('best alpha = ', best_alpha)
    print('best score = ', best_score)
    
    full_dataset = pd.concat([train, test])
    model = smf.glm(formula=model_formula, data=full_dataset, family=sm.families.NegativeBinomial(alpha=best_alpha))

    fitted_model = model.fit()
    return fitted_model    

In [15]:
iq_train_set = pd.concat([Ytrn_iq, Xtrn_iq], axis=1)
iq_test_set = pd.concat([Ytst_iq, Xtst_iq], axis=1)

print( "Iquitos")
iq_model = get_best_iq_model(iq_train_set, iq_test_set)


Iquitos
best alpha =  1e-08
best score =  7.105769230769231


In [16]:
reg_sj = RandomForestRegressor(max_depth=3, n_estimators=30, random_state=67)
reg_sj.fit(Xtrn_sj, Ytrn_sj)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=3,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=30,
                      n_jobs=None, oob_score=False, random_state=67, verbose=0,
                      warm_start=False)

In [17]:
Xtst_sj.shape

(188, 7)

In [18]:
ypred_sj = reg_sj.predict(Xtst_sj)
ypred_sj = ypred_sj.round().astype(int)
sj_pred = pd.DataFrame({'Prediction': ypred_sj, 'Actual' : Ytst_sj})

In [19]:
iq_pred = pd.DataFrame({'Prediction': iq_model.fittedvalues, 'Actual' : Ytst_iq})

In [20]:
predict_df = pd.concat([sj_pred, iq_pred], axis=0)
predict_df.loc[predict_df.Prediction < 0, 'Prediction'] = 0
predict_df.fillna(method='ffill', inplace=True)

In [21]:
def mae_calc(test, predicted):
    return (np.abs(test - predicted).sum())/len(test)

In [22]:
mae_calc(predict_df.Actual, predict_df.Prediction)

11.165590397332993

In [23]:
sub_sj = test_sj[['city','year']]
sub_iq = test_iq[['city','year']]

test_sj = test_sj.drop(['city','year'],axis=1)
test_iq = test_iq.drop(['city','year'],axis=1)

In [34]:
iq_pred_final = iq_model.predict(test_iq)
sj_pred_final = reg_sj.predict(test_sj)

iq_sub_final = pd.DataFrame({'total_cases': iq_pred_final, 'city' : sub_iq.city, 
                             'year' : sub_iq.year, 'weekofyear' : test_iq.weekofyear})
sj_sub_final = pd.DataFrame({'total_cases': sj_pred_final, 'city' : sub_sj.city, 
                             'year' : sub_sj.year, 'weekofyear' : test_sj.weekofyear})

predict_final = pd.concat([sj_sub_final, iq_sub_final], axis=0)
predict_final.total_cases = predict_final.total_cases.astype(int)
predict_final.loc[predict_final.total_cases < 0, 'total_cases'] = 0
predict_final.total_cases.fillna(method='ffill', inplace=True)

In [35]:
benchmark = pd.read_csv('data/submission_format.csv')
benchmark['total_cases'] = predict_final['total_cases'].values
benchmark.to_csv('benchmark.csv', index=False)